In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re
import pickle

In [3]:
def makelink_by_tag(tags) :
    links = []
    
    for tag in tags:
        print(tag)
        page = 1

        while True: # 페이지 끝날때까지 반복

            req = requests.get('https://kin.naver.com/tag/tagDetail.naver?tag='+tag+'&listType=answer&page='+str(page))
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')

            a = soup.select('#content > div > div.paginate3._default_pager > a')
            if len(a) == 0 :
                break

            print(page, end=' ')
            elements =  soup.select('#content > div > div.detail_list > div.question_content > ul > li > div.tit > a')

            for e in elements: # 한페이지의 질문 링크 수집
                link = 'https://kin.naver.com' + e.get('href')
                links.append(link)

            page += 1
        
        print()
        
    return links

taglink = makelink_by_tag(['식물','화분','반려식물','다육이','분갈이','식물물주기','식물관리','물주기','가지치기','선인장','식물병충해','식물키우기','몬스테라','여인초','알로카시아','공기정화식물','플랜테리어','분갈이방법','과습','스칸디아모스','다육식물','화초','산세베리아','개운죽','화분'])


식물
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 
화분
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 
반려식물
1 2

In [2]:
def makelink_by_category(categories) :
    links = []

    for c in categories :

        print(c)
        page = 0

        while True:

            page += 1
        
            if page >= 31:
                break
            print(page, end=' ')

            req = requests.get('https://kin.naver.com/qna/kinupList.naver?dirId='+c+'&page='+str(page))
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')

            elements =  soup.select('#au_board_list > tr > .title > a')

            for e in elements: # 한페이지의 질문 링크 수집
                link = 'https://kin.naver.com' + e.get('href')
                links.append(link)

    return links

# categorylink = makelink_by_category(['111607']) # 식물학 카테고리
categorylink = makelink_by_category(['80604']) # 원예

80604
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

In [4]:
# def save_link(filepath):
#     with open(filepath, 'wb') as lf:
#         links = taglink + categorylink
#         print('수집 링크 개수 : ', len(link))
#         links = list(set(links))
#         print('총 링크 개수 : ', len(links))
#         pickle.dump(links, lf)
        
def load_link(filepath):
    with open(filepath, 'rb') as lf:
        links = pickle.load(lf)
        print('총 링크 개수 : ', len(links))
    return links

In [5]:
filepath = '../data/KinNaverLink.txt'
# save_link(filepath)
links = load_link(filepath)

총 링크 개수 :  14880


In [6]:
# crawling test
def get_soup(link):
    req = requests.get(link)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def crawling_title(soup):
    title = soup.select_one('.title').text.strip()
#     print('제목 :', title)
    return title

def crawling_content(soup):
    try :
        content = ''
        findall = soup.find("div", {"class": "c-heading__content"}).find_all(text=True)
        
        for x in findall:
            temp = re.sub(r"\s+", " ", x)
            content += temp + ' '
        content = content.strip()
        
    except AttributeError:
        content = ''
        
#     print('내용 :', content)
    return content

def crawling_answer_num(soup):
    # 답변개수는 저장 안함
    try:
        answer_num = int(soup.find("em", {"class": "_answerCount"}).find(text=True))
    except:
        answer_num = 0
#     print("답변개수 :", answer_num)
    return answer_num
        

def crawling_answers(soup, answer_num):
    answers = []
    i = 0
    while True:
        i += 1
        if i > answer_num:
            break
        answer = ''
        try :
            selector = '#answer_'+str(i)+' div.se-main-container div.se-text p'
            find_all = soup.select(selector)
            find_all[0]
            
        except Exception as ex:
            try:
                find_all = soup.select_one('#answer_'+str(i)+" > div._endContents").find("div",{"class": "_endContentsText"}).find_all(text=True)
            except Exception as ex2:
                continue  # 삭제된 답변인 경우
        
        
        for x in find_all:
            x = re.sub(r"\s+", " ", x.text)
            if x != '\u200b' and 'http' not in x:
                answer += (x + ' ')
            
        answer = answer.strip()
        answers.append(answer)
        
#     print(answers)
    return answers


In [105]:
# test
link = 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80604&docId=286675834'
soup = get_soup(link)
title = crawling_title(soup)
content = crawling_content(soup)
answer_num = crawling_answer_num(soup)
if answer_num >= 1:
    answers = crawling_answers(soup, answer_num)

제목 : 스투키 새순??? 어떻게 해야하나요! (사진첨부)
내용 : 거의 2년정도 키우고 있는 선물받은 스투키입니다! 중간에 분갈이도 진행하였고 첨에 새순이 조금씩 올라오나 싶더니 어느순간 보니 이렇게 많이 ㅠㅠ 올라왔더라구요!  여기저기 찾아보니 옮겨심어야 한다는 분들도 있고, 그냥 두라는 분들도 있고.......  어떻게 해야하나요~~~? 그리고 저 아이들이 나중에 동그랗게 말려서 제가 아는 스투키처럼 단단해지나요!??!!?!?
답변개수 : 2
1
ex :  list index out of range
ex2 :  'NoneType' object has no attribute 'find'
2
ex :  list index out of range
['화분이 식물에 비해서 좁으니까 분갈이를 해주면 스투키가 자라는데 도움이 됩니다. 분갈이하기 키우기']


In [7]:
links = categorylink

In [7]:
results = []
def crawling(links):
    cnt = 0
    for link in links:
        print(cnt, end=' ')
        json_dic = dict()
        
        try:
            soup = get_soup(link)
            title = crawling_title(soup)
            content = crawling_content(soup)
            answer_num = crawling_answer_num(soup)
            answers = []
            if answer_num >= 1:
                answers = crawling_answers(soup, answer_num)
            
            json_dic['id'] = cnt
            json_dic['link'] = link
            json_dic['title'] = title
            json_dic['content'] = content
            json_dic['answers'] = answers
            
            cnt+=1
            results.append(json_dic)
            
        except Exception as e:
            print(e)
            print('except no : ', cnt, '/ link : ', link)
            return 


In [8]:
crawling(links)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [134]:
results = results[10:]

In [135]:
len(results)

14880

In [9]:
with open('../data/QuestionDB.json', 'w', encoding='UTF-8') as jf:
    json.dump(results, jf, indent=2, ensure_ascii=False)

In [140]:
documents = []
for x in results:
    
    title = x['title']
    content = x['content']
    answer = ''
    for ans in x['answers']:
        answer += ans + ' '
        
    document = title + ' ' + content + ' ' + answer
    document = document.strip()
    documents.append(document)


In [145]:
train = documents[:11904]
test = documents[11904:]

In [148]:
with open('../data/trainset.txt', 'wb') as lf:
    pickle.dump(train, lf)

with open('../data/testset.txt', 'wb') as lf:
    pickle.dump(test, lf)